In [158]:
import pandas as pd
import numpy as np
import scipy
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn import neighbors
from sklearn.model_selection import cross_val_score
import statsmodels.formula.api as smf
from statsmodels.sandbox.regression.predstd import wls_prediction_std

#### Data

The data is obtained from UCI's machine learning repository and contain information on concrete strenght based on various factors.

Link: https://archive.ics.uci.edu/ml/datasets/Concrete+Compressive+Strength

In [153]:
data = pd.read_csv("Concrete_Data.csv")

In [154]:
data.shape

(1030, 9)

In [155]:
data.columns

Index(['Cement (component 1)(kg in a m^3 mixture)',
       'Blast Furnace Slag (component 2)(kg in a m^3 mixture)',
       'Fly Ash (component 3)(kg in a m^3 mixture)',
       'Water  (component 4)(kg in a m^3 mixture)',
       'Superplasticizer (component 5)(kg in a m^3 mixture)',
       'Coarse Aggregate  (component 6)(kg in a m^3 mixture)',
       'Fine Aggregate (component 7)(kg in a m^3 mixture)', 'Age (day)',
       'Concrete compressive strength(MPa, megapascals) '],
      dtype='object')

In [14]:
data.head()

,cement,Blast Furnace Slag (component 2)(kg in a m^3 mixture),Fly Ash (component 3)(kg in a m^3 mixture),Water (component 4)(kg in a m^3 mixture),Superplasticizer (component 5)(kg in a m^3 mixture),Coarse Aggregate (component 6)(kg in a m^3 mixture),Fine Aggregate (component 7)(kg in a m^3 mixture),Age (day),"Concrete compressive strength(MPa, megapascals)"
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


In [156]:
data.columns =["cement","slag","ash","water","superplasticizer","coarse","fine","age","strength"]

In [201]:
data["age_squared"] = data.age **2

In [202]:
data.describe()

,cement,slag,ash,water,superplasticizer,coarse,fine,age,strength,age_squared
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,281.167864,73.895825,54.188350,181.567282,6.204660,972.918932,773.580485,45.662136,35.817961,6071.594175
std,104.506364,86.279342,63.997004,21.354219,5.973841,77.753954,80.175980,63.169912,16.705742,20195.954706
min,102.000000,0.000000,0.000000,121.800000,0.000000,801.000000,594.000000,1.000000,2.330000,1.000000
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,23.710000,49.000000
50%,272.900000,22.000000,0.000000,185.000000,6.400000,968.000000,779.500000,28.000000,34.445000,784.000000
75%,350.000000,142.950000,118.300000,192.000000,10.200000,1029.400000,824.000000,56.000000,46.135000,3136.000000
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,82.600000,133225.000000


In [200]:
knn = neighbors.KNeighborsRegressor(n_neighbors=15, weights='distance')
X = data[["cement","slag","ash","water","superplasticizer","coarse","fine","age"]]
Y = data["strength"]
knn.fit(X, Y)
score = cross_val_score(knn, X, Y, cv=3)
print("Weighted Accuracy: %0.2f (+/- %0.2f)" % (score.mean(), score.std() * 2))

Weighted Accuracy: 0.41 (+/- 0.15)


In [244]:
knn = neighbors.KNeighborsRegressor(n_neighbors=15, weights='distance')
X = data[["cement","slag","ash","water","superplasticizer","age","age_squared"]]
Y = data["strength"]
knn.fit(X, Y)
score = cross_val_score(knn, X, Y, cv=10)
print("Weighted Accuracy: %0.2f (+/- %0.2f)" % (score.mean(), score.std() * 2))

Weighted Accuracy: 0.64 (+/- 0.52)


In [207]:
# OLS
linear_formula = "strength ~ cement+slag+ash+water+superplasticizer+age+age_squared"
lm = smf.ols(formula=linear_formula, data = data).fit()


In [208]:
lm.params

Intercept           21.043748
cement               0.106594
slag                 0.087160
ash                  0.064582
water               -0.206101
superplasticizer     0.179027
age                  0.353197
age_squared         -0.000816
dtype: float64

In [209]:
lm.pvalues

Intercept            1.620559e-09
cement              1.534229e-147
slag                 1.108314e-84
ash                  1.974587e-23
water                7.084681e-31
superplasticizer     9.690505e-03
age                 5.686029e-148
age_squared          2.005539e-92
dtype: float64

In [210]:
lm.rsquared

0.74308624170634974